<a href="https://colab.research.google.com/github/Liambeck99/open-source-guide/blob/18f-pages/ECG_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECG Classification

## Project Goal
The goal of this project is to create a deep neural network able to classify ECG signals and cardiac arrhythmia with a high degree of accuracy. Alongside this goal is the aim to explore the implications of introducing simulated data into the training sets in hopes of improving the overall accuracy of the model.

The model created and used for this project is based off of the model preseneted in the Yildrim20 paper :
https://www.sciencedirect.com/science/article/abs/pii/S016926072031573X


#### Firstly import all of the neccesary librarys

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
import scipy.io

from google.colab import files
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, BatchNormalization, LeakyReLU, Dropout, LSTM, Flatten, Dense 
from keras.utils import to_categorical


#### Import the datafiles for training and testing


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv("/content/drive/My Drive/ECG/ammendedData/ammendedData.csv").values
labels = pd.read_csv("/content/drive/My Drive/ECG/ammendedData/ammendedLabels.csv").values

In [ ]:
X_train = np.array(data[0:250])
X_train = X_train[..., None]
Y_train = np.array(labels[0:250])

X_test = np.array(data[251:301])
X_test = X_test[..., None]
Y_test = np.array(labels[251:301])

X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)

In [ ]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
Y_train = encoder.fit_transform(Y_train)
Y_test = encoder.fit_transform(Y_test)

[[0 0 1 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 1 0 0]
 [0 0 1 0]]


#### Initialise variables needed for the CNN

In [ ]:
nb_epoch = 1 # Make larger later

batch_size = 250

# more ...

print(X_train.shape)
print(X_test.shape)

(200, 5000, 1)
(30, 5000, 1)


#### Create the model

In [ ]:
# Initialise model
model = Sequential()

# 'Input' layer chunk
model.add( Conv1D( filters=64, kernel_size=21, strides=11, input_shape=[5000, 1] ) )
model.add( MaxPooling1D( pool_size=2 ) )
model.add( BatchNormalization() )
model.add( LeakyReLU( alpha=0.1 ) )
model.add( Dropout( rate=0.3 ) )

# Hidden layers
model.add( Conv1D( filters=64, kernel_size=7, strides=1, input_shape=[64, 226] ) )
model.add( MaxPooling1D( pool_size=2 ) )
model.add( BatchNormalization() )

model.add( Conv1D( filters=128, kernel_size=5, strides=1, input_shape=[64, 110] ) )
model.add( MaxPooling1D( pool_size=2 ) )

model.add( Conv1D( filters=256, kernel_size=13, strides=1, input_shape=[128, 53] ) )
model.add( Conv1D( filters=512, kernel_size=7, strides=1, input_shape=[256, 41] ) )
model.add( Dropout( rate=0.3 ) )

model.add( Conv1D( filters=256, kernel_size=9, strides=1, input_shape=[512, 35] ) )
model.add( MaxPooling1D( pool_size=2 ) )

# 'Output' layer chunk
model.add( LSTM( units=128, return_sequences=True) )
model.add( Flatten() )
model.add( Dense( units=64, activation='relu' ) )
model.add( Dense( units=4, activation='softmax' ) )


model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_48 (Conv1D)           (None, 453, 64)           1408      
_________________________________________________________________
max_pooling1d_32 (MaxPooling (None, 226, 64)           0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 226, 64)           256       
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 226, 64)           0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 226, 64)           0         
_________________________________________________________________
conv1d_49 (Conv1D)           (None, 220, 64)           28736     
_________________________________________________________________
max_pooling1d_33 (MaxPooling (None, 110, 64)          

#### Compile the Model

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.002)
model.compile(
    loss = "categorical_crossentropy",
    optimizer=opt,
    metrics=['accuracy']
)

#### Fit model on training data

In [ ]:
model.fit(
    X_train, Y_train,
    validation_data=(X_test, Y_test),
    epochs=30,
)

Epoch 1/30
8/8 [==============================] - 7s 653ms/step - loss: 0.9903 - accuracy: 0.5880 - val_loss: 0.9740 - val_accuracy: 0.6000
Epoch 2/30
8/8 [==============================] - 5s 571ms/step - loss: 0.9947 - accuracy: 0.5880 - val_loss: 0.9659 - val_accuracy: 0.6000
Epoch 3/30
8/8 [==============================] - 4s 561ms/step - loss: 0.9767 - accuracy: 0.5880 - val_loss: 0.9628 - val_accuracy: 0.6000
Epoch 4/30
8/8 [==============================] - 4s 557ms/step - loss: 0.9703 - accuracy: 0.5880 - val_loss: 0.9607 - val_accuracy: 0.6000
Epoch 5/30
8/8 [==============================] - 4s 559ms/step - loss: 0.9775 - accuracy: 0.5880 - val_loss: 0.9734 - val_accuracy: 0.6000
Epoch 6/30
8/8 [==============================] - 4s 560ms/step - loss: 0.9833 - accuracy: 0.5800 - val_loss: 0.9784 - val_accuracy: 0.6000
Epoch 7/30
8/8 [==============================] - 4s 557ms/step - loss: 0.9753 - accuracy: 0.5880 - val_loss: 0.9887 - val_accuracy: 0.6000
Epoch 8/30
8/8 [====

#### Evaluate the model on test data

In [ ]:
score = model.evaluate(
    X_test,
    Y_test
)

32/32 [==============================] - 4s 114ms/step - loss: 0.9807 - accuracy: 0.6046
